In [1]:
%matplotlib ipympl
from matplotlib import pylab as plt
import matplotlib
import molpx

import os
import sys
import glob
from itertools import cycle

import numpy as np
import pandas as pd

from os.path import join

import seaborn as sns

import mdtraj as md
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform

import pyemma

import random

from os.path import join

from collections import OrderedDict
from collections import defaultdict

import unicodedata


print("Python version:\n{}\n".format(sys.version))
print("matplotlib version: {}".format(matplotlib.__version__))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("seaborn version: {}".format(sns.__version__))

Python version:
3.6.8 |Anaconda custom (64-bit)| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]

matplotlib version: 3.1.0
pandas version: 0.24.2
numpy version: 1.16.4
seaborn version: 0.9.0


In [28]:
indir = './Trajectories_all_old'
topfile =  indir+'/open10_dry.parm7'
from glob import glob
traj_list = glob(indir+'/*.nc')

In [29]:
ref1 = md.load('./Trajectories_all/open10_dry.rst7', top='./Trajectories_all/open10_dry.parm7')
ref2 = md.load('./Trajectories_all/closed10_dry.rst7', top='./Trajectories_all/closed10_dry.parm7')
ref3 = md.load_pdb('./cluster_centre_2/0.pdb')
ref4 = md.load_pdb('./cluster_centre_2/1.pdb')
ref5 = md.load_pdb('./cluster_centre_2/2.pdb')
ref6 = md.load_pdb('./cluster_centre_2/3.pdb')
ref7 = md.load_pdb('./cluster_centre_2/4.pdb')
ref8 = md.load_pdb('./cluster_centre_2/5.pdb')
ref9 = md.load_pdb('./cluster_centre_2/6.pdb')
ref10 = md.load_pdb('./cluster_centre_2/7.pdb')
ref11 = md.load_pdb('./cluster_centre_2/8.pdb')
ref12 = md.load_pdb('./cluster_centre_2/9.pdb')
    

In [30]:
selection = ref1.topology.select_expression('backbone')
print(selection)

[atom.index for atom in topology.atoms if atom.is_backbone]


In [31]:
BB=[atom.index for atom in ref1.topology.atoms if atom.is_backbone]


In [32]:
feat = pyemma.coordinates.featurizer(topfile)
feat.add_minrmsd_to_ref(ref1, atom_indices=BB)
feat.add_minrmsd_to_ref(ref2, atom_indices=BB)


feat.describe()

['minrmsd to frame 0 of <mdtraj.Trajectory with 1 frames, 2808 atoms, 192 residues, and unitcells at 0x7f606ac2e780>, subset of atoms  ',
 'minrmsd to frame 0 of <mdtraj.Trajectory with 1 frames, 2808 atoms, 192 residues, and unitcells at 0x7f606f341978>, subset of atoms  ']

In [33]:
feat.dimension()

2

In [34]:
inp = pyemma.coordinates.source(traj_list, feat)

In [35]:
print('number of trajectories =',inp.number_of_trajectories())
print('trajectory length =',inp.trajectory_length(0),'frames')
print('trajectory time =',(inp.trajectory_length(0)*50/1000),'ns')
print('trajectory time step =',(inp.trajectory_length(0)*50)/inp.trajectory_length(0),'ps')
print('number of dimension =',inp.dimension())

S=0
for i in range(0, inp.number_of_trajectories()):
    #print(inp.trajectory_length(i))
    S += inp.trajectory_length(i)
print('total length =',S,'frames')
T=S*50/1000000
print('total simulated time =',T,'μs')

number of trajectories = 10
trajectory length = 40200 frames
trajectory time = 2010.0 ns
trajectory time step = 50.0 ps
number of dimension = 2
total length = 398311 frames
total simulated time = 19.91555 μs


In [36]:
tica_obj = pyemma.coordinates.tica(data=inp, lag=20, var_cutoff=1)
print('TICA dimension ', tica_obj.dimension())

TICA dimension  2


In [37]:
tica_obj.cumvar

array([0.56416682, 1.        ])

In [38]:
Y = tica_obj.get_output()

In [39]:
mpx_wdg_box = molpx.visualize.FES(traj_list,
                                 #MD_trajfiles,
                                 topfile,
                                 Y,
                                 #Y,
                                 nbins=50,
                                 #proj_idxs=[1,2],
                                 proj_labels='TIC',
                                 #n_overlays=5,
                                )
mpx_wdg_box

KeyboardInterrupt: 

In [ ]:
#plt.figure()
pyemma.plots.plot_free_energy(np.vstack(Y)[:, 0], np.vstack(Y)[:, 1]);

In [ ]:
mpx_wdg_box = molpx.visualize.traj(traj_list, 
                                   topfile, 
                                   Y,
                                   #Y_filenames,                                          
                                   plot_FES = False,                                        
                                   dt = 0.01, tunits='ns',    
                                   #traj_selection = 1,
                                   #sharey_traj=False,
                                   #max_frames=100,
                                   proj_idxs=[0, 1],
                                   panel_height=2,    
                                   #proj_labels='TIC'
                                  )
mpx_wdg_box

## Regspace Clustering

In [13]:
clustering = pyemma.coordinates.cluster_regspace(Y,dmin=0.25)

In [14]:
def plot_labels(ax=None):
    #if ax is None:
        #ax = gca()
    for i in range(0,len(clustering.clustercenters)):
        plt.text(clustering.clustercenters[i][0]+0.05, clustering.clustercenters[i][1]+0.05, i, fontsize=10, color='black')


In [15]:
#plt.figure()
pyemma.plots.plot_free_energy(np.vstack(Y)[:, 0], np.vstack(Y)[:, 1])
cc_x = clustering.clustercenters[:, 0]
cc_y = clustering.clustercenters[:, 1]
plt.plot(cc_x, cc_y, linewidth=0, marker='o', markersize=5, color='black')
#plot_labels()

FigureCanvasNbAgg()

In [20]:
labels = np.concatenate(clustering.dtrajs)
mapping = defaultdict(lambda : [])
for i, label in enumerate(labels):
    mapping[label].append(i)
for i in range(0,len(clustering.clustercenters)):
    print('size of cluster %d: %d structures' % (i, len(mapping[i])))


size of cluster 0: 34 structures
size of cluster 1: 9 structures
size of cluster 2: 66 structures
size of cluster 3: 1611 structures
size of cluster 4: 286 structures
size of cluster 5: 1538 structures
size of cluster 6: 97 structures
size of cluster 7: 918 structures
size of cluster 8: 77 structures
size of cluster 9: 234 structures
size of cluster 10: 832 structures
size of cluster 11: 50 structures
size of cluster 12: 777 structures
size of cluster 13: 3073 structures
size of cluster 14: 2785 structures
size of cluster 15: 1828 structures
size of cluster 16: 2226 structures
size of cluster 17: 1895 structures
size of cluster 18: 144 structures
size of cluster 19: 355 structures
size of cluster 20: 545 structures
size of cluster 21: 262 structures
size of cluster 22: 224 structures
size of cluster 23: 465 structures
size of cluster 24: 591 structures
size of cluster 25: 22 structures
size of cluster 26: 2225 structures
size of cluster 27: 1007 structures
size of cluster 28: 954 struc

In [21]:
data_sample, geoms = molpx.generate.sample(traj_list, topfile, clustering, n_geom_samples=200, n_points=200)

In [22]:
clustering.max_centers

269

In [23]:
FILE = open( './cluster_centre_1/clustering.log', "w" )

#trajs={}

for i in range(0,clustering.max_centers):
    print('size of cluster %d: %d structures' % (i, len(mapping[i])))
    print('size of cluster %d: %d structures' % (i, len(mapping[i])), file=FILE)
    geoms[i].save('./cluster_centre_1/{}.pdb'.format(i))

size of cluster 0: 34 structures
size of cluster 1: 9 structures
size of cluster 2: 66 structures
size of cluster 3: 1611 structures
size of cluster 4: 286 structures
size of cluster 5: 1538 structures
size of cluster 6: 97 structures
size of cluster 7: 918 structures
size of cluster 8: 77 structures
size of cluster 9: 234 structures
size of cluster 10: 832 structures
size of cluster 11: 50 structures
size of cluster 12: 777 structures
size of cluster 13: 3073 structures
size of cluster 14: 2785 structures
size of cluster 15: 1828 structures
size of cluster 16: 2226 structures
size of cluster 17: 1895 structures
size of cluster 18: 144 structures
size of cluster 19: 355 structures
size of cluster 20: 545 structures
size of cluster 21: 262 structures
size of cluster 22: 224 structures
size of cluster 23: 465 structures
size of cluster 24: 591 structures
size of cluster 25: 22 structures
size of cluster 26: 2225 structures
size of cluster 27: 1007 structures
size of cluster 28: 954 struc

size of cluster 235: 782 structures
size of cluster 236: 866 structures
size of cluster 237: 943 structures
size of cluster 238: 1023 structures
size of cluster 239: 1356 structures
size of cluster 240: 119 structures
size of cluster 241: 31 structures
size of cluster 242: 994 structures
size of cluster 243: 456 structures
size of cluster 244: 11 structures
size of cluster 245: 666 structures
size of cluster 246: 108 structures
size of cluster 247: 142 structures
size of cluster 248: 115 structures
size of cluster 249: 176 structures
size of cluster 250: 310 structures
size of cluster 251: 152 structures
size of cluster 252: 97 structures
size of cluster 253: 4 structures
size of cluster 254: 17 structures
size of cluster 255: 8 structures
size of cluster 256: 14 structures
size of cluster 257: 85 structures
size of cluster 258: 1 structures
size of cluster 259: 484 structures
size of cluster 260: 189 structures
size of cluster 261: 4 structures
size of cluster 262: 54 structures
size 

## K-mean clustering - key clusters

In [13]:
n_clusters = 10

In [14]:
clustering = pyemma.coordinates.cluster_kmeans(Y,k=n_clusters)

01-07-18 23:12:14 pyemma.coordinates.clustering.kmeans.KmeansClustering[3] INFO     Algorithm did not reach convergence criterion of 1e-05 in 10 iterations. Consider increasing max_iter.


In [21]:
def plot_labels(ax=None):
    #if ax is None:
        #ax = gca()
    for i in range(0,len(clustering.clustercenters)):
        plt.text(clustering.clustercenters[i][0]+0.07, clustering.clustercenters[i][1]+0.07, 
                 i, fontsize=10, color='white')


In [22]:
pyemma.plots.plot_free_energy(np.vstack(Y)[:, 0], np.vstack(Y)[:, 1])
cc_x = clustering.clustercenters[:, 0]
cc_y = clustering.clustercenters[:, 1]
plt.plot(cc_x, cc_y, linewidth=0, marker='o', markersize=5, color='white')
plot_labels()

FigureCanvasNbAgg()

In [17]:
data_sample, geoms = molpx.generate.sample(traj_list, topfile, clustering, n_geom_samples=1000, n_points=200)

In [18]:
labels = np.concatenate(clustering.dtrajs)
mapping = defaultdict(lambda : [])
for i, label in enumerate(labels):
    mapping[label].append(i)
for i in range(0,len(clustering.clustercenters)):
    print('size of cluster %d: %d structures' % (i, len(mapping[i])))

size of cluster 0: 78613 structures
size of cluster 1: 19614 structures
size of cluster 2: 48854 structures
size of cluster 3: 10044 structures
size of cluster 4: 47784 structures
size of cluster 5: 44164 structures
size of cluster 6: 100748 structures
size of cluster 7: 15429 structures
size of cluster 8: 14999 structures
size of cluster 9: 18062 structures


In [35]:
len(clustering.clustercenters)

10

In [36]:
trajs={}

for i in range(0,len(clustering.clustercenters)):
    print('size of cluster %d: %d structures' % (i, len(mapping[i])))
    geoms[i].save('./cluster_centre_2/{}.pdb'.format(i))

size of cluster 0: 16022 structures
size of cluster 1: 87064 structures
size of cluster 2: 50547 structures
size of cluster 3: 16892 structures
size of cluster 4: 36577 structures
size of cluster 5: 17799 structures
size of cluster 6: 49546 structures
size of cluster 7: 58811 structures
size of cluster 8: 44723 structures
size of cluster 9: 20330 structures
